<img src="Images/HSP2.png" />
This Jupyter Notebook Copyright 2017 by RESPEC, INC.  All rights reserved.

$\textbf{HSP}^{\textbf{2}}\ \text{and}\ \textbf{HSP2}\ $ Copyright 2017 by RESPEC INC. and released under this [License](LegalInformation/License.txt)

# View RCHRES Data Saved in HDF5 File

This notebook displays RCHRES data from the specified HDF5 file.

The Notebook  demonstrates how monthly and annual summary results can be calculated via Pandas. It uses aggregation methods 'last' (for the last value at the end of each report interval) and
'sum (which sums the timeseries within each report interval). There are many other aggregation methods available (such as max, min, and mean).  The user can create their own aggragation function as needed.

This Notebook also demonstrates plotting capability in Pandas. These plots can be enhanced with titles, x & y axis labels, legends, line styles, point styles, etc. and saved in various formats (SVG, PDF, etc.).  Matplotlib (the basis for these Pandas plots) can be used to make publication quality plots. See Tutorial 5 for more information.

### Required Python imports and configurations

In [ ]:
import os
import site
site.addsitedir(os.getcwd().rsplit('\\',1)[0] + '\\')  # adds your path to the HSP2 software.

import shutil
import numpy as np
import pandas as pd
pd.options.display.max_rows    = 16
pd.options.display.max_columns = 20
pd.options.display.float_format = '{:.2f}'.format  # display 2 digits after the decimal point

from matplotlib import pyplot as plt

# replace with "%matplotlib notebook" for interactive plots
%matplotlib inline

import HSP2tools
HSP2tools.versions()

### Set HDF5 filename and paths to the data

In [ ]:
hdfname = os.path.join('TutorialData', 'Tutorial.h5')
segment = 'R005'          # Reaches rid  

hydr = '/RESULTS/RCHRES_' + segment + '/HYDR'

Names of the possible HYDR timeseries created by HSP$^2$ - user may not save all these during a simulation run.

In [ ]:
state_variables = ['VOL', 'DEP', 'SAREA', 'USTAR',  'TAU', 'RO', 'O1', 'O2', 'O3', 'O4', 'O5']
fluxes          = ['IVOL', 'PRSUPY',  'VOLEV', 'ROVOL', 'OVOL1', 'OVOL2', 'OVOL3', 'OVOL4', 'OVOL15']

Read what data is actually in the HDF5 file. Update the lists of column names to reflect only the available data.

In [ ]:
columns = pd.read_hdf(hdfname, hydr).columns

state_variables = [v for v in state_variables if v in columns]
fluxes          = [v for v in fluxes          if v in columns]

### Read HYDR  State Variables from the HDF5 file
##### Computed Results, Generally Hourly

In [ ]:
sv = pd.read_hdf(hdfname, hydr)[state_variables]
sv.index.names = [segment]
sv

##### Monthly Report
These values are rounded to 3 decimal places for convenience.

In [ ]:
pd.options.display.float_format = '{:.3f}'.format 
sv.resample('M').last()

### Calculated results - not stored in HDF5 since it is simple to post process these results

##### Conversion factors (from internal ft$^3$ to acre-ft)

In [ ]:
VFACT = 43560.0   #$162
AFACT = 43560.0   #$164

#### LEN (Length) of Reach (Necessary for the calculations)

In [ ]:
# First - need length of this reach - get it from HDF5 file
with pd.get_store(hdfname, mode='r') as store:
    x = store['RCHRES/HYDR/PARAMETERS']
    LEN = x.loc[segment, 'LEN'] 
LEN = LEN * 5280.0

##### Calculate Additional State Variables from simulation data saved in the HDF5 file
Note: The calculations are undefined when the flow or volumn is zero, so the  fillna() function replaces NaNs with zeros, where needed.

These calculations need the data associated with SAREA, VOL, and RO, so the calculations are conditional.

In [ ]:
if 'SAREA' in sv and 'VOL' in sv and 'RO' in sv:
    sv['TWID']   = AFACT * sv['SAREA']/LEN
    sv['AVDEP']  = (sv['VOL']  / sv['SAREA']).fillna(0.0)
    sv['HRAD']   = (sv['AVDEP'] * sv['TWID']) / (2.0 * sv['AVDEP'] + sv['TWID']).fillna(0.0)
    sv['AVVEL']  = ((LEN * sv['RO'] / sv['VOL']) / AFACT).fillna(0.0)
    sv['AVSECT'] = AFACT * sv['VOL'] / LEN

Update the state_variables list to reflect the actual available data. The data is also rounded to 2 decimal places for convenience.

In [ ]:
pd.options.display.float_format = '{:.2f}'.format  # display 2 digits after the decimal point

state_variables = ['VOL', 'DEP', 'AVDEP', 'TWID', 'HRAD', 'SAREA', 'AVVEL', 'AVSECT', 'USTAR', 'TAU', 'RO']
state_variables = [v for v in state_variables if v in sv]

sv[state_variables].resample('M').last()

Plot this data

In [ ]:
_ = sv.plot(subplots=True, figsize=[20,48])

### Read HYDR Flux Data from the HDF5 file

##### Raw Hourly Data

In [ ]:
flx = pd.read_hdf(hdfname, hydr)[fluxes]
flx.index.names = [segment]
flx

Plot this data.

In [ ]:
_ = flx.plot(figsize=[20,40], subplots=True)

##### Monthly Report
This data is rounded to s decimal places for convenience.

In [ ]:
pd.options.display.float_format = '{:.3f}'.format

flx.resample('M').sum()

##### Annual Report

Annual reports follow calendar year by default, however you can easily select any month to start. These values are rounded to 2 decimal places for convenience.

In [ ]:
flx.resample('A').sum()  # Annual report, January start month